# tatForge Shipping Document Extraction

Extract structured data from shipping stem documents using **GPT-4o vision** via BAML.

## Document Types Covered

| Document | Structure | Challenge |
|----------|-----------|----------|
| **CBH Shipping Stem** | Multi-port tables with merged header cells | Port name in colored header |
| **Queensland Bulk Terminals** | Vertical label-value layout | Multiple columns per page |
| **GrainCorp Shipping Stem** | Very dense tabular data | High density, many rows skipped |

---

## Setup

In [ ]:
import os
import sys
import json
import base64
from pathlib import Path

# Add project root to path
sys.path.insert(0, '..')

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Import dependencies
import baml_py
from baml_client import b
from tatforge.flows import file_to_pages

# Verify setup
api_key = os.getenv("OPENAI_API_KEY")
print(f"OPENAI_API_KEY: {'Set' if api_key else 'NOT SET'}")
print(f"BAML client: {hasattr(b, 'ExtractDocumentFieldsFromImage')}")

# Helper function for extraction
def extract_from_pdf(pdf_path: Path, prompt: str, page_num: int = 0):
    """Extract data from a specific page of a PDF."""
    pdf_bytes = pdf_path.read_bytes()
    pages = file_to_pages(pdf_path.name, pdf_bytes)
    
    if page_num >= len(pages):
        raise ValueError(f"Page {page_num} not found. PDF has {len(pages)} pages.")
    
    page_image = pages[page_num].image
    image_b64 = base64.b64encode(page_image).decode("utf-8")
    baml_image = baml_py.Image.from_base64("image/png", image_b64)
    
    result = b.ExtractDocumentFieldsFromImage(
        document_image=baml_image,
        extraction_prompt=prompt
    )
    
    try:
        return json.loads(result)
    except json.JSONDecodeError:
        return result

print("Setup complete!")

---

## Example 1: CBH Shipping Stem

**File:** `CBH Shipping Stem 26092025.pdf`

**Structure:** Multi-port document with tables grouped by port. Port name appears in a colored merged header cell above each table section.

**Challenge:** The port name (GERALDTON, KWINANA, ALBANY, ESPERANCE) is in the first merged cell of each section, not in the data rows.

In [ ]:
# CBH Shipping Stem - with port name in schema
cbh_pdf = Path("../pdfs/CBH Shipping Stem 26092025.pdf")

cbh_prompt = """
This is a CBH Daily Ship Roster document with multiple port sections.

IMPORTANT: Each section has a PORT NAME in a colored header row (e.g., "GERALDTON", "KWINANA", "ALBANY", "ESPERANCE").
You MUST include the port name for each shipment.

Extract ALL shipments from ALL port sections. Return as JSON:

{
    "document_date": "date from 'As of' field",
    "shipments": [
        {
            "port": "port name from the colored section header (GERALDTON/KWINANA/ALBANY/ESPERANCE)",
            "vna_number": "VNA # column",
            "vessel_name": "Vessel Name",
            "client": "Client/Exporter name",
            "eta_date": "ETA date",
            "etc_date": "ETC (Estimated Time of Commencement) date",
            "etd_date": "ETD (Estimated Time of Departure) date",
            "volume_tonnes": "Volume in tonnes (number only)",
            "commodity": "Wheat/Barley/Canola/Lupins etc",
            "loading_status": "Completed/Commenced/blank",
            "other_ports": "if vessel visits other ports (e.g., 'ex ALBANY')"
        }
    ],
    "port_totals": {
        "GERALDTON": "total tonnes",
        "KWINANA": "total tonnes",
        "ALBANY": "total tonnes",
        "ESPERANCE": "total tonnes"
    }
}

CRITICAL: Do not skip any rows. Extract EVERY shipment from EVERY port section.
"""

print(f"Processing: {cbh_pdf.name}")
print("Extracting...")

cbh_result = extract_from_pdf(cbh_pdf, cbh_prompt, page_num=0)

print("\nExtracted Data:")
print("=" * 60)
print(json.dumps(cbh_result, indent=2))

In [ ]:
# Validate CBH extraction - count shipments by port
if isinstance(cbh_result, dict) and "shipments" in cbh_result:
    shipments = cbh_result["shipments"]
    print(f"Total shipments extracted: {len(shipments)}")
    
    # Count by port
    port_counts = {}
    for s in shipments:
        port = s.get("port", "UNKNOWN")
        port_counts[port] = port_counts.get(port, 0) + 1
    
    print("\nShipments by port:")
    for port, count in sorted(port_counts.items()):
        print(f"  {port}: {count}")
    
    # Check for missing ports
    expected_ports = {"GERALDTON", "KWINANA", "ALBANY", "ESPERANCE"}
    found_ports = set(port_counts.keys())
    missing = expected_ports - found_ports
    if missing:
        print(f"\nWARNING: Missing ports: {missing}")
else:
    print("Extraction did not return expected structure")

---

## Example 2: Queensland Bulk Terminals

**File:** `document (1).pdf`

**Structure:** Vertical label-value format with 2 shipment columns per page. Multiple pages.

**Challenge:** Data is arranged vertically (field names on left, values in columns), not in typical table rows.

In [ ]:
# Queensland Bulk Terminals
qbt_pdf = Path("../pdfs/document (1).pdf")

qbt_prompt = """
This is a Queensland Bulk Terminals shipping document with a VERTICAL layout.

The document shows 2 shipments per page in columns. Field labels are on the left,
and values are in columns to the right.

Extract ALL shipments from this page. Return as JSON:

{
    "last_updated": "from 'Last updated on' field",
    "shipments": [
        {
            "slot_reference": "Unique Slot Reference Number",
            "vessel_name": "Name of ship",
            "port": "Port name (e.g., Brisbane)",
            "nomination_date": "Date at which nomination was received",
            "nomination_time": "Time at which nomination was received",
            "acceptance_date": "Date at which nomination was accepted",
            "acceptance_time": "Time at which nomination was accepted",
            "eta_from_date": "Date of ETA of Ship From",
            "eta_to_date": "Date of ETA of Ship To",
            "loading_commencement_date": "Date ETA of Grain Loading Commencement",
            "etd_date": "Date of ETD of Ship",
            "exporter": "Exporter name",
            "quantity_tonnes": "Quantity in tonnes (number only)",
            "commodity": "Commodity type (e.g., QBT Wheat, QBT Sorghum)",
            "loading_status": "Loading commenced or completed",
            "loading_completed_date": "Date Loading Completed (if applicable)"
        }
    ]
}

Extract BOTH columns on this page (2 shipments).
"""

print(f"Processing: {qbt_pdf.name}")

# Extract from first page
print("\nPage 1:")
qbt_page1 = extract_from_pdf(qbt_pdf, qbt_prompt, page_num=0)
print(json.dumps(qbt_page1, indent=2))

In [ ]:
# Extract ALL pages from QBT document
qbt_bytes = qbt_pdf.read_bytes()
qbt_pages = file_to_pages(qbt_pdf.name, qbt_bytes)

print(f"Total pages: {len(qbt_pages)}")

all_qbt_shipments = []

for i in range(len(qbt_pages)):
    print(f"Processing page {i+1}/{len(qbt_pages)}...")
    try:
        result = extract_from_pdf(qbt_pdf, qbt_prompt, page_num=i)
        if isinstance(result, dict) and "shipments" in result:
            for s in result["shipments"]:
                s["source_page"] = i + 1
                all_qbt_shipments.append(s)
            print(f"  Found {len(result['shipments'])} shipments")
    except Exception as e:
        print(f"  Error: {e}")

print(f"\nTotal QBT shipments extracted: {len(all_qbt_shipments)}")

---

## Example 3: GrainCorp Shipping Stem (Dense Document)

**File:** `shipping-stem-2025-11-13.pdf`

**Structure:** Very dense tabular data organized by Month and Port. Many columns, many rows.

**Challenge:** Document is extremely dense. Simple extraction skips many rows. Need strategies:
1. Extract by port section
2. Use more specific prompts
3. Process page by page with row counting

In [ ]:
# GrainCorp Shipping Stem - Dense document
graincorp_pdf = Path("../pdfs/shipping-stem-2025-11-13.pdf")

# Strategy 1: Simple extraction (baseline - expect missing rows)
graincorp_simple_prompt = """
This is a GrainCorp Shipping Stem document with dense tabular data.

Extract ALL shipments. Return as JSON:

{
    "document_date": "Thursday, 13 November 2025",
    "shipments": [
        {
            "month": "Month (e.g., Nov 25, Dec 25)",
            "port": "Port name",
            "slot_reference": "Unique Slot Reference Number",
            "exporter": "Exporter code (GCOP, BUNGE, CARG, etc.)",
            "vessel_name": "Name Of Ship",
            "eta_date": "Date ETA of Ship",
            "loading_commencement_date": "Date of Grain Loading Commencement",
            "etd_date": "Date ETD of Ship",
            "status": "Status (Accepted/COMMENCED)",
            "commodity": "Commodity type",
            "total_tonnes": "Total tonnes (number only)"
        }
    ],
    "grand_total": "Grand Total tonnes"
}

IMPORTANT: This document is VERY DENSE. Count every row carefully.
Do NOT skip any rows. Extract EVERY single shipment.
"""

print(f"Processing: {graincorp_pdf.name}")
print("\nStrategy 1: Simple extraction (Page 1)")
print("="*60)

graincorp_simple = extract_from_pdf(graincorp_pdf, graincorp_simple_prompt, page_num=0)

if isinstance(graincorp_simple, dict) and "shipments" in graincorp_simple:
    print(f"Shipments found: {len(graincorp_simple['shipments'])}")
    print(f"Grand total: {graincorp_simple.get('grand_total', 'N/A')}")
else:
    print("Unexpected result format")
    print(json.dumps(graincorp_simple, indent=2)[:1000])

In [ ]:
# Strategy 2: Extract by PORT SECTION for better accuracy
graincorp_port_prompt = """
This is a GrainCorp Shipping Stem. Extract shipments for ONE PORT SECTION at a time.

Focus on the "{port}" section only. Return as JSON:

{{
    "port": "{port}",
    "shipments": [
        {{
            "month": "Month",
            "slot_reference": "Reference number",
            "exporter": "Exporter code",
            "vessel_name": "Ship name",
            "eta_date": "ETA date",
            "etd_date": "ETD date",
            "commodity": "Commodity",
            "total_tonnes": "Tonnes"
        }}
    ],
    "port_total": "Total for this port section"
}}

Extract EVERY row in the {port} section. Count carefully.
"""

ports = ["Mackay", "Gladstone", "Fisherman Islands", "Carrington", "Port Kembla", "Geelong", "Portland"]

print("Strategy 2: Extract by port section (Page 1)")
print("="*60)

all_port_results = {}
for port in ports[:3]:  # Test with first 3 ports
    prompt = graincorp_port_prompt.format(port=port)
    print(f"\nExtracting {port}...")
    result = extract_from_pdf(graincorp_pdf, prompt, page_num=0)
    
    if isinstance(result, dict):
        shipments = result.get("shipments", [])
        port_total = result.get("port_total", "N/A")
        print(f"  Shipments: {len(shipments)}, Total: {port_total}")
        all_port_results[port] = result

In [ ]:
# Strategy 3: Row-by-row extraction with explicit counting
graincorp_detailed_prompt = """
This is a VERY DENSE GrainCorp Shipping Stem document.

I need you to extract data with EXTREME PRECISION. Follow these rules:

1. Count EVERY row in EVERY table section
2. Do NOT skip rows even if data is repeated or similar
3. Include rows with "TBA" vessel names
4. Include rows with "Blank" or "(blank)" values
5. Include rows for non-grain items (Cement, Woodchip, BERTH WORKS)

Return JSON with this structure:

{
    "extraction_stats": {
        "total_rows_counted": "actual count of ALL data rows",
        "rows_by_month": {"Nov 25": N, "Dec 25": N, ...}
    },
    "shipments": [
        {
            "row_number": "sequential row number for verification",
            "month": "Month",
            "port": "Port",
            "slot_ref": "Slot reference",
            "exporter": "Exporter",
            "vessel": "Vessel name",
            "commodity": "Commodity",
            "tonnes": "Tonnes"
        }
    ]
}

CRITICAL: The document shows approximately 100+ rows across 3 pages.
If you extract fewer than 50 rows from page 1, you are missing data.
"""

print("Strategy 3: Detailed extraction with row counting (Page 1)")
print("="*60)

detailed_result = extract_from_pdf(graincorp_pdf, graincorp_detailed_prompt, page_num=0)

if isinstance(detailed_result, dict):
    stats = detailed_result.get("extraction_stats", {})
    shipments = detailed_result.get("shipments", [])
    print(f"Reported row count: {stats.get('total_rows_counted', 'N/A')}")
    print(f"Actual shipments extracted: {len(shipments)}")
    print(f"\nRows by month: {json.dumps(stats.get('rows_by_month', {}), indent=2)}")
else:
    print("Unexpected format")
    print(str(detailed_result)[:500])

In [ ]:
# Extract ALL pages from GrainCorp with detailed prompt
graincorp_bytes = graincorp_pdf.read_bytes()
graincorp_pages = file_to_pages(graincorp_pdf.name, graincorp_bytes)

print(f"GrainCorp document has {len(graincorp_pages)} pages")
print("="*60)

all_graincorp_shipments = []
page_stats = []

for i in range(len(graincorp_pages)):
    print(f"\nProcessing page {i+1}/{len(graincorp_pages)}...")
    result = extract_from_pdf(graincorp_pdf, graincorp_detailed_prompt, page_num=i)
    
    if isinstance(result, dict):
        shipments = result.get("shipments", [])
        for s in shipments:
            s["source_page"] = i + 1
            all_graincorp_shipments.append(s)
        
        stats = result.get("extraction_stats", {})
        page_stats.append({
            "page": i + 1,
            "reported_rows": stats.get("total_rows_counted"),
            "extracted": len(shipments)
        })
        print(f"  Extracted: {len(shipments)} shipments")

print("\n" + "="*60)
print(f"TOTAL shipments extracted: {len(all_graincorp_shipments)}")
print(f"\nPage statistics:")
for ps in page_stats:
    print(f"  Page {ps['page']}: {ps['extracted']} extracted (reported: {ps['reported_rows']})")

---

## Results Comparison

Compare extraction results across document types.

In [ ]:
# Summary of all extractions
print("Extraction Summary")
print("="*60)

results = [
    ("CBH Shipping Stem", len(cbh_result.get("shipments", [])) if isinstance(cbh_result, dict) else 0),
    ("Queensland Bulk Terminals", len(all_qbt_shipments)),
    ("GrainCorp Shipping Stem", len(all_graincorp_shipments)),
]

for doc, count in results:
    print(f"{doc}: {count} shipments")

print("\n" + "="*60)
print("\nNOTE: GrainCorp document is very dense.")
print("Expected ~100+ shipments across 3 pages.")
print("If extraction is significantly lower, consider:")
print("  1. Using ColPali for visual similarity search")
print("  2. Breaking document into smaller sections")
print("  3. Using table-specific extraction tools")

---

## Next Steps

For dense documents like GrainCorp:

1. **ColPali Indexing**: Index all pages with ColPali embeddings for visual search
   ```bash
   cocoindex setup
   cocoindex update
   ```

2. **Semantic Search**: Query specific shipments by port, commodity, or vessel
   ```python
   query_embedding = query_to_colpali_embedding.eval("Portland wheat shipments")
   ```

3. **Targeted Extraction**: Extract from search results instead of whole document

### Resources

- [BAML Documentation](https://docs.boundaryml.com)
- [CocoIndex Documentation](https://cocoindex.io/docs)
- [ColPali Paper](https://arxiv.org/abs/2407.01449)